In [11]:
import torch
import numpy as np
from math import sin, cos
from typing import List, Dict

#FOR_GPU
device = torch.device('cuda')
device = torch.device('cpu')
RACHEL1 = "<s> I hate cheese . </s>"
RACHEL2 = "<s> Do you like cheese ? </s>"
RACHEL3 = "<s> Where are the balloons ? </s>"
RACHEL4 = "<s> A monkey ate my sandwich . </s>"

def getVocabDicts(sents: List[str]) -> (Dict[str, int], Dict[int, str]):
    vocabDict = dict()
    inv_vocab = dict()
    i = 0
    for sent in sents:
        for token in sent.split():
            if token not in vocabDict:
                vocabDict[token] = i
                i = i + 1
    vocabDict["<unk>"] = i
    
    for word in vocabDict:
        inv_vocab[vocabDict[word]] = word
    
    return (vocabDict, inv_vocab)
    

SENTENCE = "<s> I LIKE PIE VERY MUCH PIE . </s>"
SENTENCE2 = "<s> WE ARE YOUNG . </s>"
SENTENCE3 = "<s> LET US TEST HOW GOOD THIS IS . </s>"
SENTENCE4 = "<s> TO SILENCE ALL THE HATERS OUT THERE . </s>"
SENTENCE5 = "<s> AND FOR GLORY , FREEDOM , FREE PIE AND YOUNG LOVERS . </s>"
SENTENCE6 = "<s> AND OF COURSE A HARD BOILED EGG IS ESSENTIAL . </s>"
DIM_EMB = 512
torch.manual_seed(7)

mysents = [SENTENCE, SENTENCE2, SENTENCE3, SENTENCE4, SENTENCE5, SENTENCE6]
rachel_sents = [RACHEL1, RACHEL2, RACHEL3, RACHEL4]
all_sents = rachel_sents + mysents
vocab, inv_vocab = getVocabDicts(all_sents)

DICT_SIZE = len(vocab)

def getBatch(sents: List[str], vocab: Dict[str, int]) -> torch.Tensor:
    #Get max sentence length
    DICT_SIZE = len(vocab)
    maxLen = 0
    for sent in sents:
        maxLen = max(maxLen, len(sent.split()))
    oneHotBatch = np.zeros((len(sents), maxLen, DICT_SIZE), dtype=float)
    for i in range(len(sents)):
        curr_sent = sents[i].split()
        for j in range(len(curr_sent)):
            word = curr_sent[j]
            oneHotBatch[i][j][vocab[word]] = 1
    return torch.as_tensor(oneHotBatch, dtype=torch.float32, device=device)

batch_tst = getBatch(all_sents, vocab)

#oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

#for i in range(len(SENTENCE.split())):
#    word = SENTENCE.split()[i]
#    oneHotSent[i][vocab[word]] = 1

#oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
class FUCKYOU(torch.nn.Module):
    def __init__(self, vocab_size, max_sent_len, heads=8, head_dim=512):
        super().__init__()
        self.DICT_SIZE = vocab_size
        self.batch_size = max_sent_len #@TODO CHANGE FIX THIS IS WRONG
        self.transformerHeadConcat = 512 #@TODO CHANGE
        self.heads = heads
        self.HEAD_DIM = head_dim
        
        # In the transformer paper, the Wq, Wk and Wv are square 512x512
        # and the number of heads is 512/64
        
        #TODO add biases because i don't have them
        Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, self.HEAD_DIM), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wq = torch.nn.Parameter(Wq_skeleton, requires_grad=True)

        Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, self.HEAD_DIM), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wk = torch.nn.Parameter(Wk_skeleton, requires_grad=True)

        Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, self.HEAD_DIM), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wv = torch.nn.Parameter(Wv_skeleton, requires_grad=True)
        
        Wo_skeleton = torch.tensor(np.zeros((self.transformerHeadConcat, DIM_EMB), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wo = torch.nn.Parameter(Wo_skeleton, requires_grad=True)
        
        #Embeddings!
        We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.We = torch.nn.Parameter(We_skeleton, requires_grad=True)
        
        Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wff = torch.nn.Parameter(Wff_skeleton, requires_grad=True)
        Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Bff = torch.nn.Parameter(Bff_skeleton, requires_grad=True)
        
        Wproj_skeleton = torch.tensor(np.zeros((DIM_EMB, DICT_SIZE), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Wproj = torch.nn.Parameter(Wproj_skeleton, requires_grad=True)
        Bproj_skeleton = torch.tensor(np.zeros((DICT_SIZE), dtype=float), dtype=torch.float32, device=device).normal_(0, 1)
        self.Bproj = torch.nn.Parameter(Bproj_skeleton, requires_grad=True)
        
        Pos_emb = np.zeros((max_sent_len, DIM_EMB), dtype=float)
        for p in range(4): #should max_sent_len but 9 works better. Go figure
            for i in range(DIM_EMB//2):
                val = p/(10000**(2*i/DIM_EMB))
                Pos_emb[p][2*i] = sin(val)
                Pos_emb[p][2*i + 1] = cos(val)

        self.Pos_emb_var = torch.nn.Parameter(torch.tensor(Pos_emb, dtype=torch.float32, device=device), requires_grad=True)
        
        torch.nn.utils.clip_grad_norm_(self.parameters(), 5)
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.0001)

        
    
    #DATA PREPARATION DONE

    # Create self attention
    # also need to add a mask here
    # @TODO Add a Bias to all Wsz
    def add_transformer_head_with_mask(self, DIM_EMB, Batch, mask):
        Q = Batch.matmul(self.Wq)
        V = Batch.matmul(self.Wv)
        K = Batch.matmul(self.Wk)
        print("Q", Q.shape)
        num_sents = Q.shape[0] #batch_size 10
        sent_len = Q.shape[1] # 14
        dim_q = Q.shape[2] # 512
        
        print("Q before reshape", Q.shape)
        Q = Q.permute(0, 2, 1)
        V = V.permute(0, 2, 1)
        K = K.permute(0, 2, 1)
        Q = Q.reshape(num_sents, sent_len, dim_q//self.heads, -1)
        V = V.reshape(num_sents, sent_len, dim_q//self.heads, -1)
        K = K.reshape(num_sents, sent_len, dim_q//self.heads, -1)
        print("Q after reshape", Q.shape)

        tmp1 = Q.matmul(K.permute(0, 1, 3, 2))
        #tmp1 = torch.t(K).matmul(Q)

        tmp = tmp1/(64**(1/2))
        #@TODO this might be wrong for dim=-1
        softM = torch.nn.Softmax(dim=-1)
        tmp = tmp.masked_fill(mask == 0, -1e9)
        #print("AFTER MASK", tmp)
        a = softM(tmp)
        #print("AFTER SOFTMAX", a)
        z = a.matmul(V)
        #print("After Z*V", z)
        return z

    # @TODO add a bias to Wo
    def addLinearProj(self, concatTransHead, DIM_EMB):
        '''Already concatenataed all heads'''

        return concatTransHead.matmul(self.Wo)

    #Normalize
    def layerNorm(self, mat):
        #@TODO there should be a square root here somewhere
        return (mat - mat.mean(dim=1, keepdim=True)) / (mat.std(dim=1, keepdim=True))

    EPOCHS=10
    def fwd(self, oneHotTorch: torch.Tensor):
        
        Batch = oneHotTorch.matmul(self.We)
        Batch = Batch*(DIM_EMB**(1/2))
        
        #Positional embeddings here
        Batch = Batch + self.Pos_emb_var
        #Mask the batch post pos_emb
        mask1 = torch.sum(oneHotTorch, dim=2) > 0
        mask1 = mask1.reshape(Batch.shape[0], Batch.shape[1], 1).float()
        
        Batch = Batch*mask1

        #Create MASK HERE
        #
        #print(Batch.shape)
        mask = np.triu(np.ones((Batch.shape[1], Batch.shape[1])), k=1)
        mask = torch.tensor(mask, device=device) == 0

        transformer_head = self.add_transformer_head_with_mask(DIM_EMB, Batch, mask)
        postAttention = self.addLinearProj(transformer_head, DIM_EMB)

        # Add residual connection to the embeddings
        postAttention = postAttention + Batch
        #print(postAttention)

        normalizedPostAttention = self.layerNorm(postAttention)
        #print(normalizedPostAttention)
        # @TODO no dropout
        # Take the output of the attention and do a one layer FF that outputs size of
        activation = torch.nn.ReLU()
        postFF = activation(normalizedPostAttention.matmul(self.Wff) + self.Bff)

        #Residual
        preSoftmaxProj = self.layerNorm(postFF + normalizedPostAttention)

        linearLay = preSoftmaxProj.matmul(self.Wproj) + self.Bproj
        return linearLay

    #@TODO
    def getOneHot(self, truth):
        ret = torch.Tensor(self.batch_size).long() 

model = FUCKYOU(DICT_SIZE, max([len(x.split()) for x in all_sents]))
print(np.mean([len(x.split()) for x in all_sents]))
EPOCHS=1

def printSent(softmaxd, vocab, batch_onehot):
    words = [[]]
    for j in range(len(softmaxd)):
        words.append([])
        for i in range(len(softmaxd[j])):
            _, idx = torch.max(softmaxd[j][i], 0)
            hasWord, _ = torch.max(batch_onehot[j][i], 0)
            if hasWord == 1:
                words[j].append(vocab[int(idx)])
    for arr in words:
        print(" ".join(arr))
    
def getTruth(oneHotSent):
    truth = []
    for j in range(len(oneHotSent)):
        truth.append([])
        for i in range(len(oneHotSent[j])):
            _, idx = torch.max(oneHotSent[j][i], 0)
            truth[j].append(idx)
    return torch.tensor(truth, device=device).long()

for e in range(EPOCHS):
    model.optimizer.zero_grad()
    fwd = model.fwd(batch_tst)
    printSent(fwd, inv_vocab, batch_tst)
    truth = getTruth(batch_tst)
    loss = torch.nn.functional.cross_entropy(fwd.permute(0, 2, 1), truth, size_average = True, ignore_index=11)
    print("Loss: ", loss.item())
    loss.backward()
    model.optimizer.step()

    



8.9
Q torch.Size([10, 14, 512])
Q before reshape torch.Size([10, 14, 512])
Q after reshape torch.Size([10, 14, 64, 8])
K permute torch.Size([10, 14, 8, 64])


RuntimeError: Expected tensor to have size 8 at dimension 1, but got size 64 for argument #2 'batch2' (while checking arguments for bmm)